In [41]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

fig_prefix = "../../figures/2018-07-22-jw-weaselball-heatmap_"
data_prefix = "../../data/2018-07-22-jw-weaselball-heatmap_"


## Downsample

In [42]:
import csv
row_count = len(open('../../data/10-17-2018_10-48-32_long.csv').readlines())
row_count

skip_rows = []
SAMPLE = 250
for i in range(1,row_count+100):
    if (i % SAMPLE != 0):
        skip_rows.append(i)

In [43]:
df = pd.read_csv('../../data/10-17-2018_10-48-32_long.csv',skiprows = skip_rows, sep=',',index_col=False)
print(df.shape)
df.head(10)


(2764, 38)


,Time,ID,Mount_X,Mount_Y,Mount_Yaw,Pos_x,Pos_y,Pos_z,Yaw,Pitch,...,Linear_Acceleration_Z_Relative_Relative,Rotational_Velocity_X_Relative,Rotational_Velocity_Y_Relative,Rotational_Velocity_Z_Relative,Rotational_Acceleration_X_Relative,Rotational_Acceleration_Y_Relative,Rotational_Acceleration_Z_Relative,ResetID,checkCorrectness,foo
0,0 250000000,48,-0.004451,-0.002147,0.005415,-0.000118,-0.011710,0.041,0.994503,0.994503,...,-5.644830,6.165010,2.421590,-2.523650,188.67700,-240.308000,-295.56200,1,1,0
1,0 500000000,48,0.051689,0.008046,0.071437,0.045414,0.015893,0.041,1.030260,1.030260,...,-2.012200,2.633730,-4.498210,-2.804300,13.31540,-58.854400,3.88177,1,1,0
2,0 750000000,48,0.060471,0.100936,0.090021,0.071668,0.104162,0.041,-1.031680,-1.031680,...,0.292525,1.055630,-9.968790,-7.008150,-42.14480,0.120057,-46.55380,1,1,0
3,1 0,48,0.129255,0.217090,0.084609,0.134982,0.227122,0.041,1.269880,1.269880,...,2.106740,0.537420,-10.656400,-9.955800,-18.08040,49.952000,23.87510,1,1,0
4,1 250000000,48,0.172756,0.325351,0.068186,0.180226,0.334315,0.041,-0.964866,-0.964866,...,-0.698295,3.275830,-4.420060,-8.142490,-15.27630,23.848300,-16.70580,1,1,0
5,1 500000000,48,0.180819,0.382005,0.051728,0.190967,0.387883,0.041,-0.586108,-0.586108,...,2.287730,4.766040,-2.386730,0.310762,49.47900,-40.860600,9.35932,1,1,0
6,1 750000000,48,0.243173,0.435796,0.026258,0.248459,0.445986,0.041,0.864826,0.864826,...,-1.532200,3.544520,-9.905760,0.063309,-2.69782,-59.980500,4.10258,1,1,0
7,2 0,48,0.282802,0.493110,0.130821,0.292311,0.499993,0.041,0.429383,0.429383,...,-0.639483,-2.553040,-0.988287,0.552507,17.06460,2.141250,-21.74670,1,1,0
8,2 250000000,48,0.310477,0.495332,0.412288,0.321659,0.495899,0.041,0.251613,0.251613,...,2.027160,-6.356480,-1.740550,3.151190,-50.38530,22.145000,38.41300,1,1,0
9,2 500000000,48,0.363370,0.431276,0.619181,0.356708,0.425188,0.041,1.178260,1.178260,...,2.363540,-0.797101,-5.999430,6.223000,-1.26033,53.156300,4.24471,1,1,0


In [44]:
#Remove useless columns
col_del = [0,1,35,36,37][::-1]
for col in col_del:
    print(df.columns[col])
    df = df.drop(df.columns[col], axis=1)
df.head()

foo
checkCorrectness
ResetID
ID
Time


,Mount_X,Mount_Y,Mount_Yaw,Pos_x,Pos_y,Pos_z,Yaw,Pitch,Roll,Linear_Velocity_X_World,...,Linear_Velocity_Z_Relative,Linear_Acceleration_X_Relative,Linear_Acceleration_Y_Relative,Linear_Acceleration_Z_Relative_Relative,Rotational_Velocity_X_Relative,Rotational_Velocity_Y_Relative,Rotational_Velocity_Z_Relative,Rotational_Acceleration_X_Relative,Rotational_Acceleration_Y_Relative,Rotational_Acceleration_Z_Relative
0,-0.004451,-0.002147,0.005415,-0.000118,-0.011710,0.041,0.994503,0.994503,0.998918,0.228423,...,0.213091,4.232340,8.730620,-5.644830,6.16501,2.42159,-2.52365,188.6770,-240.308000,-295.56200
1,0.051689,0.008046,0.071437,0.045414,0.015893,0.041,1.030260,1.030260,-2.896540,0.076714,...,-0.213499,-0.007694,-0.159214,-2.012200,2.63373,-4.49821,-2.80430,13.3154,-58.854400,3.88177
2,0.060471,0.100936,0.090021,0.071668,0.104162,0.041,-1.031680,-1.031680,-1.969240,0.138495,...,0.339589,-0.301125,-0.498565,0.292525,1.05563,-9.96879,-7.00815,-42.1448,0.120057,-46.55380
3,0.129255,0.217090,0.084609,0.134982,0.227122,0.041,1.269880,1.269880,1.420840,0.371260,...,-0.415880,-0.715255,-1.151510,2.106740,0.53742,-10.65640,-9.95580,-18.0804,49.952000,23.87510
4,0.172756,0.325351,0.068186,0.180226,0.334315,0.041,-0.964866,-0.964866,-2.430610,-0.040189,...,0.137811,0.450414,-0.279755,-0.698295,3.27583,-4.42006,-8.14249,-15.2763,23.848300,-16.70580


In [47]:
#Clean x,y data
df_clean = df
#Clean up the data

#Shift X and Y over so that way it could be made easier to use. Currently the world reference is at the center of the "play area"
LENGTH_OF_BOX = 1.127 #This can be obtained from the .sdf file of the weazelball enclosure in gazebo

df_clean['Mount_X'] += LENGTH_OF_BOX / 2
df_clean['Mount_Y'] += LENGTH_OF_BOX / 2
df_clean.head()

,Mount_X,Mount_Y,Mount_Yaw,Pos_x,Pos_y,Pos_z,Yaw,Pitch,Roll,Linear_Velocity_X_World,...,Linear_Velocity_Z_Relative,Linear_Acceleration_X_Relative,Linear_Acceleration_Y_Relative,Linear_Acceleration_Z_Relative_Relative,Rotational_Velocity_X_Relative,Rotational_Velocity_Y_Relative,Rotational_Velocity_Z_Relative,Rotational_Acceleration_X_Relative,Rotational_Acceleration_Y_Relative,Rotational_Acceleration_Z_Relative
0,0.559049,0.561353,0.005415,-0.000118,-0.011710,0.041,0.994503,0.994503,0.998918,0.228423,...,0.213091,4.232340,8.730620,-5.644830,6.16501,2.42159,-2.52365,188.6770,-240.308000,-295.56200
1,0.615189,0.571546,0.071437,0.045414,0.015893,0.041,1.030260,1.030260,-2.896540,0.076714,...,-0.213499,-0.007694,-0.159214,-2.012200,2.63373,-4.49821,-2.80430,13.3154,-58.854400,3.88177
2,0.623971,0.664436,0.090021,0.071668,0.104162,0.041,-1.031680,-1.031680,-1.969240,0.138495,...,0.339589,-0.301125,-0.498565,0.292525,1.05563,-9.96879,-7.00815,-42.1448,0.120057,-46.55380
3,0.692755,0.780590,0.084609,0.134982,0.227122,0.041,1.269880,1.269880,1.420840,0.371260,...,-0.415880,-0.715255,-1.151510,2.106740,0.53742,-10.65640,-9.95580,-18.0804,49.952000,23.87510
4,0.736256,0.888851,0.068186,0.180226,0.334315,0.041,-0.964866,-0.964866,-2.430610,-0.040189,...,0.137811,0.450414,-0.279755,-0.698295,3.27583,-4.42006,-8.14249,-15.2763,23.848300,-16.70580


In [49]:
#Discretize x,y
NUMBER_OF_SQUARES = 400 #This should be a square number to create equal sized squares.
RESOLUTION_OF_S1 = 0.1 #This is used to discretize the yaw angle over 0 - 2*pi


mappingBoxConstant = (NUMBER_OF_SQUARES ** (1/2.0)) / (LENGTH_OF_BOX)
for index, row in df_clean.iterrows():
    df_clean.at[index, 'Mount_X'] = int(row['Mount_X'] * mappingBoxConstant)
    df_clean.at[index, 'Mount_Y'] = int(row['Mount_Y'] * mappingBoxConstant)

df_clean.head()

,Mount_X,Mount_Y,Mount_Yaw,Pos_x,Pos_y,Pos_z,Yaw,Pitch,Roll,Linear_Velocity_X_World,...,Linear_Velocity_Z_Relative,Linear_Acceleration_X_Relative,Linear_Acceleration_Y_Relative,Linear_Acceleration_Z_Relative_Relative,Rotational_Velocity_X_Relative,Rotational_Velocity_Y_Relative,Rotational_Velocity_Z_Relative,Rotational_Acceleration_X_Relative,Rotational_Acceleration_Y_Relative,Rotational_Acceleration_Z_Relative
0,9.0,9.0,0.005415,-0.000118,-0.011710,0.041,0.994503,0.994503,0.998918,0.228423,...,0.213091,4.232340,8.730620,-5.644830,6.16501,2.42159,-2.52365,188.6770,-240.308000,-295.56200
1,10.0,10.0,0.071437,0.045414,0.015893,0.041,1.030260,1.030260,-2.896540,0.076714,...,-0.213499,-0.007694,-0.159214,-2.012200,2.63373,-4.49821,-2.80430,13.3154,-58.854400,3.88177
2,11.0,11.0,0.090021,0.071668,0.104162,0.041,-1.031680,-1.031680,-1.969240,0.138495,...,0.339589,-0.301125,-0.498565,0.292525,1.05563,-9.96879,-7.00815,-42.1448,0.120057,-46.55380
3,12.0,13.0,0.084609,0.134982,0.227122,0.041,1.269880,1.269880,1.420840,0.371260,...,-0.415880,-0.715255,-1.151510,2.106740,0.53742,-10.65640,-9.95580,-18.0804,49.952000,23.87510
4,13.0,15.0,0.068186,0.180226,0.334315,0.041,-0.964866,-0.964866,-2.430610,-0.040189,...,0.137811,0.450414,-0.279755,-0.698295,3.27583,-4.42006,-8.14249,-15.2763,23.848300,-16.70580


In [51]:
#Create goal columns (x_t+1, y_t+1, z_t+1)

df_clean['Mount_X_Next'] = df_clean['Mount_X']
df_clean['Mount_Y_Next'] = df_clean['Mount_Y']
df_clean['Mount_Yaw_Next'] = df_clean['Mount_Yaw']

df_clean['Mount_X_Next'].shift(-1)
df_clean['Mount_Y_Next'].shift(-1)
df_clean['Mount_Yaw_Next'].shift(-1)

df_clean.drop(df_clean.index[len(df_clean)-1])
df_clean.head()

,Mount_X,Mount_Y,Mount_Yaw,Pos_x,Pos_y,Pos_z,Yaw,Pitch,Roll,Linear_Velocity_X_World,...,Linear_Acceleration_Z_Relative_Relative,Rotational_Velocity_X_Relative,Rotational_Velocity_Y_Relative,Rotational_Velocity_Z_Relative,Rotational_Acceleration_X_Relative,Rotational_Acceleration_Y_Relative,Rotational_Acceleration_Z_Relative,Mount_X_Next,Mount_Y_Next,Mount_Yaw_Next
0,9.0,9.0,0.005415,-0.000118,-0.011710,0.041,0.994503,0.994503,0.998918,0.228423,...,-5.644830,6.16501,2.42159,-2.52365,188.6770,-240.308000,-295.56200,9.0,9.0,0.005415
1,10.0,10.0,0.071437,0.045414,0.015893,0.041,1.030260,1.030260,-2.896540,0.076714,...,-2.012200,2.63373,-4.49821,-2.80430,13.3154,-58.854400,3.88177,10.0,10.0,0.071437
2,11.0,11.0,0.090021,0.071668,0.104162,0.041,-1.031680,-1.031680,-1.969240,0.138495,...,0.292525,1.05563,-9.96879,-7.00815,-42.1448,0.120057,-46.55380,11.0,11.0,0.090021
3,12.0,13.0,0.084609,0.134982,0.227122,0.041,1.269880,1.269880,1.420840,0.371260,...,2.106740,0.53742,-10.65640,-9.95580,-18.0804,49.952000,23.87510,12.0,13.0,0.084609
4,13.0,15.0,0.068186,0.180226,0.334315,0.041,-0.964866,-0.964866,-2.430610,-0.040189,...,-0.698295,3.27583,-4.42006,-8.14249,-15.2763,23.848300,-16.70580,13.0,15.0,0.068186


In [9]:
#one-hot encode the ID

In [54]:
#Split into training and testing
df_clean['is_train'] = np.random.uniform(0, 1, len(df_clean)) <= .85
df_clean.head()

,Mount_X,Mount_Y,Mount_Yaw,Pos_x,Pos_y,Pos_z,Yaw,Pitch,Roll,Linear_Velocity_X_World,...,Rotational_Velocity_X_Relative,Rotational_Velocity_Y_Relative,Rotational_Velocity_Z_Relative,Rotational_Acceleration_X_Relative,Rotational_Acceleration_Y_Relative,Rotational_Acceleration_Z_Relative,Mount_X_Next,Mount_Y_Next,Mount_Yaw_Next,is_train
0,9.0,9.0,0.005415,-0.000118,-0.011710,0.041,0.994503,0.994503,0.998918,0.228423,...,6.16501,2.42159,-2.52365,188.6770,-240.308000,-295.56200,9.0,9.0,0.005415,True
1,10.0,10.0,0.071437,0.045414,0.015893,0.041,1.030260,1.030260,-2.896540,0.076714,...,2.63373,-4.49821,-2.80430,13.3154,-58.854400,3.88177,10.0,10.0,0.071437,True
2,11.0,11.0,0.090021,0.071668,0.104162,0.041,-1.031680,-1.031680,-1.969240,0.138495,...,1.05563,-9.96879,-7.00815,-42.1448,0.120057,-46.55380,11.0,11.0,0.090021,True
3,12.0,13.0,0.084609,0.134982,0.227122,0.041,1.269880,1.269880,1.420840,0.371260,...,0.53742,-10.65640,-9.95580,-18.0804,49.952000,23.87510,12.0,13.0,0.084609,True
4,13.0,15.0,0.068186,0.180226,0.334315,0.041,-0.964866,-0.964866,-2.430610,-0.040189,...,3.27583,-4.42006,-8.14249,-15.2763,23.848300,-16.70580,13.0,15.0,0.068186,True


In [55]:
#Create dataframes based on split
train, test = df_clean[df_clean['is_train']==True], df_clean[df_clean['is_train']==False]
# Show the number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

('Number of observations in the training data:', 2355)
('Number of observations in the test data:', 409)


In [56]:
#print list of information of each column (sanity check)
df_clean.describe()

,Mount_X,Mount_Y,Mount_Yaw,Pos_x,Pos_y,Pos_z,Yaw,Pitch,Roll,Linear_Velocity_X_World,...,Linear_Acceleration_Z_Relative_Relative,Rotational_Velocity_X_Relative,Rotational_Velocity_Y_Relative,Rotational_Velocity_Z_Relative,Rotational_Acceleration_X_Relative,Rotational_Acceleration_Y_Relative,Rotational_Acceleration_Z_Relative,Mount_X_Next,Mount_Y_Next,Mount_Yaw_Next
count,2764.00000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,...,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.00000,2764.000000,2764.000000
mean,9.30246,9.768452,0.087080,-0.011185,0.014708,0.041001,-0.015135,-0.015135,0.009267,-0.000746,...,0.030654,0.112492,-5.912272,0.045531,0.508456,-3.732908,0.475302,9.30246,9.768452,0.087080
std,5.75445,5.706344,1.970029,0.327304,0.324579,0.000023,0.766448,0.766448,1.817753,0.261161,...,5.257134,3.620193,5.124057,3.616296,55.187324,243.167096,88.584663,5.75445,5.706344,1.970029
min,1.00000,1.000000,-3.139880,-0.508790,-0.508604,0.040972,-1.554250,-1.554250,-3.139430,-0.742676,...,-81.732600,-11.058300,-18.327700,-11.720800,-1016.500000,-2873.150000,-1364.220000,1.00000,1.000000,-3.139880
25%,4.00000,5.000000,-1.871500,-0.318218,-0.273922,0.041000,-0.640993,-0.640993,-1.558148,-0.166871,...,-1.021675,-2.307097,-9.791238,-2.369615,-14.812275,-49.036650,-15.536350,4.00000,5.000000,-1.871500
50%,9.00000,10.000000,0.443469,-0.011861,0.020609,0.041000,-0.042123,-0.042123,0.045905,0.007604,...,0.042672,0.048201,-6.039580,0.040317,0.020793,-17.291750,-0.467508,9.00000,10.000000,0.443469
75%,15.00000,15.000000,1.921998,0.292018,0.308528,0.041000,0.625453,0.625453,1.583840,0.155419,...,1.045522,2.427140,-2.378585,2.442090,16.230025,28.672925,15.400500,15.00000,15.000000,1.921998
max,18.00000,18.000000,3.141260,0.508434,0.508620,0.041975,1.546770,1.546770,3.139850,0.739979,...,196.896000,11.944500,8.565230,12.876600,732.339000,8499.230000,3039.340000,18.00000,18.000000,3.141260


In [97]:
# Create a list of the feature column's names
max_index = df_clean.columns.get_loc('Mount_X_Next')
features = df_clean.columns[:max_index]

# View features
features

Index([u'Mount_X', u'Mount_Y', u'Mount_Yaw', u'Pos_x', u'Pos_y', u'Pos_z',
       u'Yaw', u'Pitch', u'Roll', u'Linear_Velocity_X_World',
       u'Linear_Velocity_Y_World', u'Linear_Velocity_Z_World',
       u'Linear_Acceleration_X_World', u'Linear_Acceleration_Y_World',
       u'Linear_Acceleration_Z_World', u'Rotational_Velocity_X_World',
       u'Rotational_Velocity_Y_World', u'Rotational_Velocity_Z_World',
       u'Rotational_Acceleration_X_World', u'Rotational_Acceleration_Y_World',
       u'Rotational_Acceleration_Z_World', u'Linear_Velocity_X_Relative',
       u'Linear_Velocity_Y_Relative', u'Linear_Velocity_Z_Relative',
       u'Linear_Acceleration_X_Relative', u'Linear_Acceleration_Y_Relative',
       u'Linear_Acceleration_Z_Relative_Relative',
       u'Rotational_Velocity_X_Relative', u'Rotational_Velocity_Y_Relative',
       u'Rotational_Velocity_Z_Relative',
       u'Rotational_Acceleration_X_Relative',
       u'Rotational_Acceleration_Y_Relative',
       u'Rotational_Accele

In [98]:
#Train the classifier
clf = RandomForestClassifier(n_jobs=-1, verbose = 1)
clf.fit(train[features], train['Mount_X_Next'])

[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [99]:
# Apply the Classifier we trained to the test data 
preds= clf.predict(test[features])

[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


In [100]:
# View the predicted probabilities of the first 10 observations
clf.predict_proba(test[features])[0:10]

[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


array([[ 0. ,  0. ,  0. ,  0.1,  0. ,  0.1,  0. ,  0. ,  0. ,  0. ,  0.1,
         0. ,  0. ,  0.1,  0.2,  0.1,  0.2,  0.1],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.1,  0.3,  0.5,  0. ,  0.1,  0. ],
       [ 0. ,  0. ,  0.1,  0. ,  0. ,  0. ,  0. ,  0. ,  0.1,  0. ,  0.1,
         0.1,  0.1,  0.1,  0.4,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.1,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0.8,  0.1],
       [ 0.2,  0.7,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.1,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0.3,  0.1,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0.1,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.1,  0. ,  0. ,  0.3,  0.2,  0.1,  0. ,  0.1,  0. ,  0. ,  0. ,
         0.1,  0. ,  0.1,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.1,  0.1,  0. ,  0. ,  0.1,  0. ,
         0. ,  0.3,  0.3,  0.1,  0. ,  0. ,  0. ],


In [101]:
# Create confusion matrix
correct = 0
confusion = pd.crosstab(test['Mount_X_Next'], preds, rownames=['Actual'], colnames=['Predicted'])
confusion

Predicted,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0
Actual,,,,,,,,,,,,,,,,,,
1.0,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2.0,0,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3.0,0,1,18,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4.0,0,0,2,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5.0,1,0,0,1,11,2,1,1,0,0,0,0,0,0,0,0,0,0
6.0,0,0,0,2,10,9,1,1,0,0,0,0,0,0,0,0,0,0
7.0,0,0,0,2,1,1,4,1,0,1,0,0,0,0,0,0,0,0
8.0,0,1,0,0,0,0,1,12,0,1,0,1,0,0,0,0,0,0
9.0,0,0,0,0,0,0,3,0,15,2,1,0,0,0,0,0,0,0


In [102]:
confusion = confusion.values #Convert to numpy

In [103]:
numerator = float(np.sum(np.diag(confusion)))
denominator = float(np.sum(confusion))
print("Accuracy is {}".format(numerator/denominator))


Accuracy is 0.770171149144


In [104]:
# View a list of the features and their importance scores
import operator
importance = list(zip(train[features], clf.feature_importances_))
importance.sort(key=operator.itemgetter(1))
importance

[('Pos_z', 0.00035068158332297341),
 ('Linear_Velocity_Z_World', 0.00081423135845108165),
 ('Linear_Acceleration_Z_World', 0.0083477027722751625),
 ('Mount_Y', 0.011601489310486098),
 ('Rotational_Velocity_Z_Relative', 0.013332587002375218),
 ('Yaw', 0.013814517049543123),
 ('Roll', 0.014147884416802963),
 ('Linear_Acceleration_Y_World', 0.014481165156200288),
 ('Rotational_Acceleration_Y_Relative', 0.014746037559091568),
 ('Rotational_Acceleration_X_Relative', 0.014791276441044767),
 ('Pitch', 0.01521740617961451),
 ('Rotational_Acceleration_X_World', 0.015251003709049155),
 ('Linear_Velocity_Z_Relative', 0.015455304624420288),
 ('Rotational_Acceleration_Z_World', 0.015937951080429825),
 ('Linear_Velocity_Y_Relative', 0.016351589513067727),
 ('Linear_Acceleration_X_Relative', 0.016385512646360963),
 ('Rotational_Velocity_X_Relative', 0.016463955167503666),
 ('Linear_Acceleration_Y_Relative', 0.016479055978301592),
 ('Rotational_Acceleration_Z_Relative', 0.016652740084786573),
 ('Rotat